In [1]:
install.packages("caret")
install.packages("ROCR")
install.packages("randomForest")
install.packages("nnet")
install.packages("ggplot2")

Installing package into 'C:/Users/KASTU1/Documents/R/win-library/3.4'
(as 'lib' is unspecified)
also installing the dependencies 'minqa', 'nloptr', 'RcppEigen', 'lme4', 'SparseM', 'MatrixModels', 'pbkrtest', 'quantreg', 'car'



package 'minqa' successfully unpacked and MD5 sums checked
package 'nloptr' successfully unpacked and MD5 sums checked
package 'RcppEigen' successfully unpacked and MD5 sums checked
package 'lme4' successfully unpacked and MD5 sums checked
package 'SparseM' successfully unpacked and MD5 sums checked
package 'MatrixModels' successfully unpacked and MD5 sums checked
package 'pbkrtest' successfully unpacked and MD5 sums checked
package 'quantreg' successfully unpacked and MD5 sums checked
package 'car' successfully unpacked and MD5 sums checked
package 'caret' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\KASTU1\AppData\Local\Temp\Rtmpcraeco\downloaded_packages


Installing package into 'C:/Users/KASTU1/Documents/R/win-library/3.4'
(as 'lib' is unspecified)


package 'ROCR' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\KASTU1\AppData\Local\Temp\Rtmpcraeco\downloaded_packages


Installing package into 'C:/Users/KASTU1/Documents/R/win-library/3.4'
(as 'lib' is unspecified)


package 'randomForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\KASTU1\AppData\Local\Temp\Rtmpcraeco\downloaded_packages


Installing package into 'C:/Users/KASTU1/Documents/R/win-library/3.4'
(as 'lib' is unspecified)


package 'nnet' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\KASTU1\AppData\Local\Temp\Rtmpcraeco\downloaded_packages


Installing package into 'C:/Users/KASTU1/Documents/R/win-library/3.4'
(as 'lib' is unspecified)


package 'ggplot2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\KASTU1\AppData\Local\Temp\Rtmpcraeco\downloaded_packages


In [2]:
library(dplyr)
library(caret)
library(ROCR)
library(randomForest)
library(nnet)
library(ggplot2)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: ggplot2
Loading required package: gplots
Warning message:
"package 'gplots' was built under R version 3.4.4"
Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:ggplot2':

    margin

The following object is masked from 'package:dplyr':

    combine



In [12]:
setwd("C:\\Users\\KASTU1\\Desktop\\Analytics Path\\R\\Data\\Kaggle\\AllStateInsurance")
train=read.csv("Allstate_test.csv", sep="|")
head(train)

customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,...,C_previous,duration_previous,A,B,C,D,E,F,G,cost
10000001,1,0,1,12:35,OK,10002,1,0,9,...,3,9,0,0,1,1,0,0,4,543
10000001,2,0,1,12:36,OK,10002,1,0,9,...,3,9,2,1,1,3,1,3,2,611
10000002,1,0,4,12:19,PA,10003,1,1,7,...,2,15,2,0,2,3,1,2,2,691
10000002,2,0,4,12:21,PA,10003,1,1,7,...,2,15,2,0,2,3,1,2,2,695
10000003,1,0,3,17:12,AR,10004,1,0,4,...,3,1,1,0,1,1,0,2,2,628
10000003,2,0,3,17:12,AR,10004,1,0,4,...,3,1,1,0,2,1,0,2,2,625


In [4]:
# Define column classes for reading in the data sets
colClasses <- c(rep("integer", 4), "character", rep("factor", 2),
                "integer", "factor", "integer", "factor", rep("integer", 3),
                rep("factor", 2), "integer", rep("factor", 7), "integer")

In [7]:
preprocess <- function(data) {
  
  # add features
  data$plan <- paste0(data$A, data$B, data$C, data$D, data$E, data$F, data$G)  ## Concatenating all the columns from A to G
  data$hour <- as.integer(substr(data$time, 1, 2))
  data$timeofday <- as.factor(ifelse(data$hour %in% 6:15, "day",
                                     ifelse(data$hour %in% 16:18, "evening",
                                            "night")))
  data$weekend <- as.factor(ifelse(data$day %in% 0:4, "No", "Yes"))
  data$family <- as.factor(ifelse(data$group_size > 2 & data$age_youngest <
                                    25 & data$married_couple==1, "Yes", "No"))
  data$agediff <- data$age_oldest-data$age_youngest
  data$individual <- as.factor(ifelse(data$agediff==0 & data$group_size==1,
                                      "Yes", "No"))
  data$stategroup <- as.factor(ifelse(data$state %in% c("SD","ND"), "g1",
                                      ifelse(data$state %in% c("AL","WY"), "g2",
                                             ifelse(data$state %in% c("OK","ME","AR",
                                                                      "WV"), "g3",
                                                    ifelse(data$state %in% c("DC","NE","GA"),
                                                           "g5", "g4")))))
  
  # fix NA's for duration_previous and C_previous
  data$duration_previous[is.na(data$duration_previous)] <- 0
  levels(data$C_previous) <- c("1", "2", "3", "4", "none")
  data$C_previous[is.na(data$C_previous)] <- "none"
  
  # replace risk_factor NA's by predicting a value
  datanorisk <- data[is.na(data$risk_factor), ]
  datarisk <- data[!is.na(data$risk_factor), ]
  lm.fit <- lm(risk_factor ~ age_youngest*group_size+married_couple+
                 homeowner, data=datarisk)
  lm.pred <- predict(lm.fit, newdata=datanorisk)
  data$risk_factor[is.na(data$risk_factor)] <- round(lm.pred, 0)
  
  # for car_age greater than 30, "round down" to 30
  data$car_age[data$car_age > 30] <- 30
  
  return(data)
}

In [15]:
 train$hour <- as.integer(substr(train$time, 1, 2))

Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion"

In [16]:
head(train$time)

[1] 12:35 12:36 12:19 12:21 17:12 17:12
1045 Levels: 0:02 0:19 0:23 0:25 0:26 0:28 0:54 0:56 0:57 1:03 1:04 1:05 ... 9:59